## Advanced Lane Finding Project

Developed by Max for Udacity second project, Dec. 2, 2018

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll import required package and plot functions

In [27]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
    
def plotresult(image,result_img,tit,fnam='test_image.jpg'):
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(result_img, cmap='gray')
    ax2.set_title(tit, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    f.savefig(fnam)   

---
## Then, I'll compute the camera calibration using chessboard images

In [34]:
# set the numbers of rows and column on chessboard
nx = 9
ny = 6

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
count = 0
for fname in images:
    count += 1
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        # c) define 4 destination points 
        offset = 100 # offset for dst points
        img_size = (img.shape[1],img.shape[0])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])
        # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
        M = cv2.getPerspectiveTransform(src,dst)
        # e) use cv2.warpPerspective() to warp your image to a top-down view

        warped = cv2.warpPerspective(img,M,img_size,flags = cv2.INTER_LINEAR)
        
        write_name = '../output_images/Output_calibration_'+str(count)+'.jpg'
        # Save images
        #plotresult(img,warped,'calibrated Camera',write_name)
        
        cv2.waitKey(500)


---
### Example of undistorted test images

In [50]:
#fname = '../test_images/straight_lines1.jpg'
images = glob.glob('../test_images/test*.jpg')

# Step through the list and plot results 
count = 0
for fname in images:
    count += 1
    img = plt.imread(fname)

    undistored_img = cv2.undistort(img, mtx, dist, None, mtx)
    write_name = '../output_images/test_image_undistorted_'+str(count)+'.jpg'
    # Save images
    plotresult(img,undistored_img,'Undistorted image',write_name)


---
## Then, I used HLS color transforms and gradients in x-direction   

In [145]:
# Image Threshod combination using both HLS and Gredient in x direction
def thereshold_image(img, s_thresh=(170, 255), sx_thresh=(30, 100)):
    img = np.copy(img)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    total_binary = np.zeros_like(s_channel)

    total_binary[(s_binary == 1) | (sxbinary==1)] = 1    
    return total_binary

---
### Example of Color/gradient thereshold on test images

In [144]:
#fname = '../test_images/straight_lines1.jpg'
images = glob.glob('../test_images/test*.jpg')

# Step through the list and plot results 
count = 0
for fname in images:
    count += 1
    img = plt.imread(fname)

    undistored_img = cv2.undistort(img, mtx, dist, None, mtx)
    binary_result = thereshold_image(undistored_img)
    write_name = '../output_images/test_image_thereshold_'+str(count)+'.jpg'
    # Save images
    plotresult(img,binary_result,'Thereshold image',write_name)


---
## Performes perspective transform 

In [46]:
# defined fun for perspective transform
def warp_image(img):
   # Defined 4 points in straight_lines Image and used for Perspective Transformation
    #Selected points for Perspective transform
    p1 = [197+10,715]
    p2 = [588+2,452]
    p3 = [696-2,452]
    p4 = [1115,715]

    #calibration box
    img_size = (img.shape[1],img.shape[0])
    
    #print(img_size)
    # four courner
    src = np.float32([p1,p2,p3,p4])
    
    # desired destination
    dst = np.float32([[280, 750],[280,0],[1000,0],[1000,750]])
    #[p1,[p1[0],int(img_size[1]/2)],[p4[0],int(img_size[1]/2)], p4])
    #print(dst)
 
    M = cv2.getPerspectiveTransform(src,dst)
    Minv= cv2.getPerspectiveTransform(dst,src)
    
    warped = cv2.warpPerspective(img,M,img_size,flags = cv2.INTER_LINEAR)
    return warped, M, Minv

---
### Example of perspective transformation on test images

In [60]:
#fname = '../test_images/straight_lines1.jpg'
images = glob.glob('../test_images/test*.jpg')

# Step through the list and plot results 
count = 0
for fname in images:
    count += 1
    img = plt.imread(fname)

    undistored_img = cv2.undistort(img, mtx, dist, None, mtx)
    result, _ , _ = warp_image(undistored_img)
    write_name = '../output_images/test_image_perspective_transformed_'+str(count)+'.jpg'
    # Save images
    plotresult(undistored_img,result,'birds-eye view',write_name)


---
### Identifying lane-line pixels and fit their positions 

In [86]:
# Find histogram for bottom half of the image
def hist(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    image_size = img.shape
    x = int(image_size[1])
    y = int(image_size[0]/2)
    bottom_half = img[y:,:]
    #plt.imshow(bottom_half)
    # Sum across image pixels vertically - make sure to set `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half,axis=0)
    
    return histogram

# Identified lane-line pixels and fit their positions 
def find_lane_pixels(binary_warped, histogram, draw = False):
    out_img = None
    if draw == True:
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        if draw == True:
        # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ###: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if draw == True:
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        #plt.imshow(out_img)

    return leftx, lefty, rightx, righty , out_img

---
### Fit a polynominal to found lane-line pixels and find the car offset and lane curvature

In [100]:
def poly_eval(poly_fit,y_val):
    # Evaluate valuate of polyfit
    return poly_fit[0]*y_val**2 + poly_fit[1]*y_val + poly_fit[2]
    
    
def plot_polynominal(img_shape,left_fit,right_fit,col = (255,0,0),thinkn = 20):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0] )
    try:
        left_fitx = poly_eval(left_fit,ploty)
        #left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        #right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]        
        right_fitx = poly_eval(right_fit,ploty)

    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    poly_img = np.zeros((img_shape[0], img_shape[1], 3), dtype=np.uint8)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    left_pts = np.vstack((left_fitx,ploty)).astype(np.int32).T
    right_pts = np.vstack((right_fitx,ploty)).astype(np.int32).T
    
    pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx,ploty])))])
    
    total_pts = []
    total_pts = np.hstack((pts_left, pts_right))
        
    cv2.fillPoly(poly_img, np.int_([total_pts]), (0, 255, 0))    
    
    cv2.polylines(poly_img,  [left_pts],  False,  col,  thinkn)
    cv2.polylines(poly_img,  [right_pts],  False,  col,  thinkn)

    #cv2.fillPoly(poly_img, np.int_(total_pts), (0, 255, 0))
    # Colors in the left and right lane regions

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')

    return poly_img

# Find car offset

def find_car_offset(left_fit,right_fit,img_shape):
    
    y_eval = img_shape[0]
    
    lx = poly_eval(left_fit,y_eval) 
    rx = poly_eval(right_fit,y_eval)
        
    xcenter =  int(img_shape[1]/2)
    #print(lx,rx,xcenter)
    
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    
    offset = ((rx - xcenter) - (xcenter - lx))*xm_per_pix
    return offset

# Calculates the curvature of polynomial

def measure_curvature_real(poly_fit,img_shape):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0] )
    plotx = poly_eval(poly_fit,ploty)

    fit_cr = np.polyfit(ploty*ym_per_pix, plotx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = img_shape[0]-1
    
    # Calculation of R_curve (radius of curvature)
    curverad_val = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    #right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return curverad_val


---
### Example of lane-line pixels identifications for test images

In [146]:
#fname = '../test_images/straight_lines1.jpg'
images = glob.glob('../test_images/test*.jpg')

# Step through the list and undistort test images 
count = 0
for fname in images:
    count += 1
    img = plt.imread(fname)

    undistored_img = cv2.undistort(img, mtx, dist, None, mtx)
    binary_img = thereshold_image(undistored_img)
    binary_warped, _ , _ = warp_image(binary_img)
    histogram = hist(binary_warped)
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped,histogram,True)    
    left_fit = np.polyfit(lefty, leftx, deg = 2)
    right_fit = np.polyfit(righty, rightx, deg = 2)
    
    poly_image = plot_polynominal(out_img.shape,left_fit,right_fit,(255,0,255),10)
    
    lines_edges = cv2.addWeighted(out_img,.7,poly_image, 0.3, 0)


    write_name = '../output_images/identified_lane_'+str(count)+'.jpg'
    # Save images
    plotresult(img,lines_edges,'Identified lane',write_name)


---
### Use search method to find new Lane around previously identified polynominal

In [88]:
def search_around_poly(binary_warped,left_fit,right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    img_shape = binary_warped.shape
    
    #left_fit = np.array([ 2.23090058e-04, -3.90812851e-01,  4.78139852e+02])
    #right_fit = np.array([ 4.19709859e-04, -4.79568379e-01,  1.11522544e+03])
    
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds =  ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

---
## Finally, I wrote the pipline code to add all previous codes to find road lanes

In [153]:
def pipline(img, i=0, pre_left_fit=None, pre_right_fit= None, weight = 0, polyfit_outlier_threshold = 100):
    
    #undistord image
    undistored_img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Filter image using HLs and Sobel thresholding
    filtered_img = thereshold_image(undistored_img,(170, 255), (30, 100))
    
    # Change Perspective from Top
    binary_warped, M, Minv = warp_image(filtered_img)
    
    # In First itireation
    if i ==0: 
        # find histogram
        histogram = hist(binary_warped)
        #find_lane_pixels    
        leftx, lefty, rightx, righty, _ = find_lane_pixels(binary_warped,histogram)
    else:
        # Search around previous poly to find the lane pixels
        leftx, lefty, rightx, righty = search_around_poly(binary_warped, pre_left_fit, pre_right_fit)
        
    # Fit Polynoinal to found pixels 
    img_shape = binary_warped.shape
    if (len(leftx) != 0) and (len(lefty) != 0):
        left_fit0 = np.polyfit(lefty, leftx, deg = 2)
    else:
        left_fit0 = pre_left_fit
    
    if (len(rightx) != 0) and (len(righty) != 0):
        right_fit0 = np.polyfit(righty, rightx, deg = 2)
    else:
        right_fit0 = pre_right_fit
    
    if i > 10:
        left_diff = left_fit0 - pre_left_fit
        right_diff = right_fit0 - pre_right_fit

        # calculate abs sume of Nomalize the difference between identified poly and previous average polys
        c_left = sum(np.absolute(left_diff/norm_vec))
        c_right = sum(np.absolute(right_diff/norm_vec))
        
        # check if norm difference is within outlier threshold
        if (c_left > polyfit_outlier_threshold) or (c_right > polyfit_outlier_threshold):
            
            # reset and re-calculate lanes based on simple histogram
            histogram = hist(binary_warped)
            leftx, lefty, rightx, righty, _ = find_lane_pixels(binary_warped,histogram)                        
            
            if c_left < polyfit_outlier_threshold:
                left_fit = (weight) * left_fit0 + (1-weight) * pre_left_fit
            if c_right < polyfit_outlier_threshold:
                right_fit =  (weight) * right_fit0 + (1-weight) * pre_right_fit     
        else:
            left_fit = (weight) * left_fit0 + (1-weight) * pre_left_fit
            right_fit =  (weight) * right_fit0 + (1-weight) * pre_right_fit
    else:
        left_fit = left_fit0
        right_fit = right_fit0          
    
    # Calculate lane curve
    left_curverad = measure_curvature_real(left_fit,img_shape)
    right_curverad = measure_curvature_real(right_fit,img_shape)
            
    car_offset = find_car_offset(left_fit,right_fit,img_shape)
    
    ## Visualization ##
    
    poly_img = plot_polynominal(img_shape,left_fit,right_fit)
    
    
    img_size = (img.shape[1],img.shape[0])
    
    # unwarped the image to road

    unwarped_img = cv2.warpPerspective(poly_img,Minv,img_size,flags = cv2.INTER_LINEAR)

    
    lines_edges = cv2.addWeighted(unwarped_img,.6,img, 0.7, 0)
   
    # Put Car offset Text
    text_car_offset = 'car offset = '+str(int(car_offset*100)) + ' cm'
    x = 500
    h = 710
    cv2.putText(lines_edges,text_car_offset, (x, h), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255))
    
    # Put Car curvature Text
    text_left_curverad = 'Left-curve rad = '+str(int(left_curverad)) + ' m'
    x = 50
    h = 100
    cv2.putText(lines_edges,text_left_curverad, (x, h), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255))
    text_right_curverad = 'Right-curve rad = '+str(int(right_curverad)) + ' m'
    x = 800
    h = 100
    cv2.putText(lines_edges,text_right_curverad, (x, h), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255))

    # color conversion for generating plots and vids
    final_img = cv2.cvtColor(lines_edges,cv2.COLOR_BGR2RGB)

    #, left_curverad, right_curverad, offset
    return final_img, left_fit0, right_fit0


---
### Example of Pipline

In [126]:
#fname = '../test_images/straight_lines1.jpg'
images = glob.glob('../test_images/test*.jpg')

# Step through the list and undistort test images 
pre_left_fit = None 
pre_right_fit = None

count = 0
for fname in images:
    count += 1
    img = plt.imread(fname)
    plt.imshow
    final_result,_, _ = pipline(img,0,pre_left_fit, pre_right_fit)

    write_name = '../output_images/Final_result_'+str(count)+'.jpg'
    # Save images
    cv2.imwrite(write_name,final_result)


---
### Read/Write files for the Project Video

In [134]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Red the project video as images
def load_video_imgs(file_name='..\project_video.mp4'):
    images=[]
    count=0       
    clip = VideoFileClip(file_name)
    images = [frame for frame in clip.iter_frames(progress_bar=True)]    
    return images

# Wirte the video from proceessed images
def make_video(images, max_index, fps=25, size=None, is_color=True, format="XVID", outvid='image_video.mp4'):
        from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
        fourcc = VideoWriter_fourcc(*format)
        vid = None
        for index in range(0,max_index):
            img = images[index]
            if vid is None:
                if size is None:
                    size = img.shape[1], img.shape[0]
                vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
            if size[0] != img.shape[1] and size[1] != img.shape[0]:
                img = resize(img, size)
            vid.write(img)
        vid.release()
        return vid

---
### Read the Project Video

In [125]:
clip = VideoFileClip("../project_video.mp4")
all_raw_images = load_video_imgs()

100%|████████████████████████████████████▉| 1260/1261 [00:06<00:00, 193.63it/s]


---
### Loop through all video images and find proceesed images 

In [ ]:
# Number of Frames in Project vidoe
max_index0 = 1260

# Set the moving average to average past indetified lanes
Moving_ave_win = 10 

# Set the outlier threshold to reset the search method arounf polunomial and use basic identification method
polyfit_outlier_threshold = 40

# Initilize the right and left fir
left_fit_hist = []
right_fit_hist = []
new_left = []
new_right = []
proceesed_images = []

del proceesed_images
c_left = np.zeros(max_index0+1)
c_right = np.zeros(max_index0+1)

norm_vec = np.array([5*10**-5 , 0.1, 60])

proceesed_images = np.copy(all_raw_images[0:max_index0])

# Loop through all video images and find proceesed images 
for x in range(0, max_index0):
    case_img = proceesed_images[x]
    if x == 0:
        final_img, pre_left_fit, pre_right_fit = pipline(case_img,0)
    else:        
        final_img, left_fit0, right_fit0 = pipline(case_img,x,pre_left_fit, pre_right_fit,weight= 1/ave_win)
        
        # use moving average to keep history of past n-fitted polynomials
        if x < Moving_ave_win :
            left_fit_hist.append(left_fit0)
            right_fit_hist.append(right_fit0)
        
        else:
            left_diff = left_fit0 - pre_left_fit
            right_diff = right_fit0 - pre_right_fit
            


            c_left[x] = sum(np.absolute(left_diff/norm_vec))
            c_right[x] = sum(np.absolute(right_diff/norm_vec))

            # add the found polynomial to the history if 
            # onlyu it is not an outlaier 
                        
            if c_left[x] < polyfit_outlier_threshold :
                new_left = left_fit_hist[1:]
                new_left.append(left_fit0)
                left_fit_hist = new_left  
                
            if c_right[x] < polyfit_outlier_threshold : 
                new_right = right_fit_hist[1:]
                new_right.append(right_fit0)
                right_fit_hist = new_right
                
        pre_left_fit = np.average(left_fit_hist,axis=0)
        pre_right_fit = np.average(right_fit_hist,axis=0)
    
    proceesed_images[x] = final_img    

---
### Write the result as a Video

In [151]:
vid = make_video(proceesed_images, max_index=max_index0, fps=25, size=None,
               is_color=True, format="XVID", outvid = '../test_videos_output/Result_project_vid2.mp4')